# Pro Mode 단계별 테스트 노트북

## 워크플로우
```
1. 사용자 입력 → 2. 증강 → 3. 플롯 생성 → 4. 에셋 생성 → 5. Kling 영상화 → 6. 합성
```

## 데이터 구조 (Pro Mode vs General Mode)

### General Mode
```
characters.json  → 캐릭터 정보
plot.json        → 씬 스크립트
layout.json      → 렌더링용 변환 데이터 ← Pro에서 불필요
```

### Pro Mode (layout.json 없음!)
```
characters.json  → 캐릭터 정보 (seed, voice_id, appearance)
plot.json        → 씬 + 에셋 URL 모두 포함
  ├── mode: "pro"
  ├── title, bgm_prompt
  ├── characters: [{char_id, name, description}]  ← 변수 치환용
  └── scenes: [
        {
          scene_id, 
          start_frame_prompt,   ← 시작 이미지 프롬프트
          end_frame_prompt,     ← 종료 이미지 프롬프트  
          text, speaker,
          duration_ms,          ← max(5000, tts_duration)
          tts_duration_ms,      ← 실제 TTS 길이
          audio_url,            ← TTS 파일 경로
          start_image_url,      ← 시작 이미지 경로
          end_image_url,        ← 종료 이미지 경로
          video_url             ← Kling 영상 경로
        }
      ]
```

## Pro 모드 핵심 규칙
1. **씬 연결**: scene_N.end_frame == scene_N+1.start_frame (연속성!)
2. **TTS**: 5초 이내 목표, 초과 시 영상 정지 후 TTS 완료 대기
3. **Duration**: `max(5000ms, tts_duration_ms)`
4. **영상**: Kling AI로 start→end 5초 영상 생성
5. **이미지 재사용**: scene_N+1.start_image = scene_N.end_image (동일 프롬프트면)

## 0. 환경 설정

In [1]:
import sys
import os
import json
from pathlib import Path

# Add backend to path
backend_path = Path.cwd().parent
if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))

# Change to backend directory for proper .env loading
os.chdir(backend_path)
print(f"Working directory: {os.getcwd()}")

# Load environment
from dotenv import load_dotenv
load_dotenv()

# Import settings
from app.config import settings

print(f"GEMINI_API_KEY: {'✅ Set' if settings.GEMINI_API_KEY else '❌ Not set'}")
print(f"MINIMAX_API_KEY: {'✅ Set' if settings.MINIMAX_API_KEY else '❌ Not set'}")
print(f"ELEVENLABS_API_KEY: {'✅ Set' if settings.ELEVENLABS_API_KEY else '❌ Not set'}")
print(f"TTS_PROVIDER: {settings.TTS_PROVIDER}")
print(f"OUTPUT_DIR: {settings.OUTPUT_DIR}")

Working directory: /Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI/backend
GEMINI_API_KEY: ✅ Set
MINIMAX_API_KEY: ✅ Set
ELEVENLABS_API_KEY: ✅ Set
TTS_PROVIDER: minimax
OUTPUT_DIR: /Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI/backend/app/data/outputs


In [2]:
# Test Run ID 설정
import uuid
from datetime import datetime

TEST_RUN_ID = f"pro_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
OUTPUT_DIR = settings.OUTPUT_DIR / TEST_RUN_ID
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Test Run ID: {TEST_RUN_ID}")
print(f"Output Dir: {OUTPUT_DIR}")

Test Run ID: pro_test_20251125_110348
Output Dir: /Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI/backend/app/data/outputs/pro_test_20251125_110348


## 1. 사용자 입력

In [3]:
# 테스트용 사용자 입력
USER_PROMPT = "카피바라가 온천에서 휴식을 취하다가 친구 오리를 만나는 이야기"

# Pro 모드 설정
SPEC = {
    "mode": "pro",
    "prompt": USER_PROMPT,
    "num_cuts": 3, 
    "art_style": "지브리 스타일 애니메이션",
    "music_genre": "peaceful lo-fi",
    "review_mode": True
}

print(f"사용자 프롬프트: {USER_PROMPT}")
print(f"씬 개수: {SPEC['num_cuts']}")
print(f"예상 영상 길이: {SPEC['num_cuts'] * 5}초")

사용자 프롬프트: 카피바라가 온천에서 휴식을 취하다가 친구 오리를 만나는 이야기
씬 개수: 3
예상 영상 길이: 15초


## 2. 프롬프트 증강 (Enhance)

In [ ]:
from app.utils.prompt_enhancer import enhance_prompt

# Pro 모드용 증강
enhanced = enhance_prompt(USER_PROMPT, mode="pro")

print("=== 증강 결과 ===")
print(f"Enhanced Prompt: {enhanced.get('enhanced_prompt', USER_PROMPT)[:200]}...")
print(f"Art Style: {enhanced.get('suggested_art_style')}")
print(f"Music Genre: {enhanced.get('suggested_music_genre')}")
print(f"Num Cuts: {enhanced.get('suggested_num_cuts')}")
print(f"Num Characters: {enhanced.get('suggested_num_characters')}")
print(f"Narrative Tone: {enhanced.get('suggested_narrative_tone')}")
print(f"Plot Structure: {enhanced.get('suggested_plot_structure')}")
print(f"Reasoning: {enhanced.get('reasoning')}")

## 3. 플롯 생성 (Characters + Plot)

### 핵심 규칙
- 각 씬: `scene_id`, `start_frame_prompt`, `end_frame_prompt`, `text`, `speaker`
- **씬 연결**: scene_N.end_frame_prompt == scene_N+1.start_frame_prompt

In [5]:
from app.utils.plot_generator import generate_plot_pro_mode

# Pro 모드 플롯 생성
characters_path, plot_path = generate_plot_pro_mode(
    run_id=TEST_RUN_ID,
    prompt=USER_PROMPT,
    num_cuts=SPEC["num_cuts"]
)

print(f"Characters: {characters_path}")
print(f"Plot: {plot_path}")

Characters: app/data/outputs/pro_test_20251125_110348/characters.json
Plot: app/data/outputs/pro_test_20251125_110348/plot.json


In [ ]:
# 생성된 캐릭터 확인
with open(characters_path, 'r', encoding='utf-8') as f:
    characters_data = json.load(f)

print("=== 캐릭터 ===")
for char in characters_data.get('characters', []):
    desc = char.get('description') or 'N/A'
    desc_preview = desc[:50] if desc != 'N/A' else 'N/A'
    print(f"  - {char['name']} ({char['char_id']}): {desc_preview}...")
    print(f"    voice_id: {char.get('voice_id')}, seed: {char.get('seed')}")

In [ ]:
# 생성된 플롯 확인
with open(plot_path, 'r', encoding='utf-8') as f:
    plot_data = json.load(f)

print(f"=== 플롯: {plot_data.get('title')} ===")
print(f"Mode: {plot_data.get('mode')}")
print(f"BGM: {plot_data.get('bgm_prompt')}")
print(f"\n씬 개수: {len(plot_data.get('scenes', []))}")

for scene in plot_data.get('scenes', []):
    print(f"\n--- {scene['scene_id']} ---")
    print(f"  Start: {scene.get('start_frame_prompt', 'N/A')[:80]}...")
    print(f"  End:   {scene.get('end_frame_prompt', 'N/A')[:80]}...")
    print(f"  Text:  {scene.get('text', 'N/A')}")
    print(f"  Speaker: {scene.get('speaker')}")

In [ ]:
# 씬 연결 검증: scene_N.end == scene_N+1.start
print("=== 씬 연결 검증 ===")
scenes = plot_data.get('scenes', [])
all_connected = True

for i in range(len(scenes) - 1):
    current_end = scenes[i].get('end_frame_prompt', '')
    next_start = scenes[i + 1].get('start_frame_prompt', '')
    
    is_connected = current_end == next_start
    status = "✅" if is_connected else "❌"
    
    print(f"{status} {scenes[i]['scene_id']} → {scenes[i+1]['scene_id']}")
    if not is_connected:
        all_connected = False
        print(f"   End:   {current_end[:60]}...")
        print(f"   Start: {next_start[:60]}...")

print(f"\n전체 연결 상태: {'✅ 모두 연결됨' if all_connected else '❌ 연결 끊김 있음'}")

## 4. 에셋 생성

### 4.1 TTS 생성
- 각 씬 텍스트 → 음성 파일
- **5초 이내 목표**, 초과 시 duration 조정

In [ ]:
# TTS 클라이언트 초기화
if settings.TTS_PROVIDER == "minimax" and settings.MINIMAX_API_KEY:
    from app.providers.tts.minimax_client import MiniMaxTTSClient
    tts_client = MiniMaxTTSClient(
        api_key=settings.MINIMAX_API_KEY,
        group_id=settings.MINIMAX_GROUP_ID
    )
    print("✅ MiniMax TTS 클라이언트 초기화")
elif settings.ELEVENLABS_API_KEY:
    from app.providers.tts.elevenlabs_client import ElevenLabsClient
    tts_client = ElevenLabsClient(api_key=settings.ELEVENLABS_API_KEY)
    print("✅ ElevenLabs TTS 클라이언트 초기화")
else:
    from app.providers.tts.stub_client import StubTTSClient
    tts_client = StubTTSClient()
    print("⚠️ Stub TTS 클라이언트 사용 (테스트 모드)")

In [ ]:
from mutagen.mp3 import MP3
import shutil

# 오디오 출력 디렉토리
audio_dir = OUTPUT_DIR / "audio"
audio_dir.mkdir(exist_ok=True)

# 캐릭터 voice_id 매핑
char_voice_map = {char['char_id']: char.get('voice_id', 'default') 
                  for char in characters_data.get('characters', [])}

print("=== TTS 생성 ===")
tts_results = []

for scene in plot_data.get('scenes', []):
    scene_id = scene['scene_id']
    text = scene.get('text', '')
    speaker = scene.get('speaker', 'narrator')
    voice_id = char_voice_map.get(speaker, 'default')
    
    if not text:
        print(f"⚠️ {scene_id}: 텍스트 없음, 스킵")
        tts_results.append({
            'scene_id': scene_id,
            'audio_path': None,
            'duration_ms': 5000  # 기본 5초
        })
        continue
    
    # TTS 생성
    output_filename = f"{scene_id}_voice.mp3"
    try:
        audio_path = tts_client.generate_speech(
            text=text,
            voice_id=voice_id,
            output_filename=output_filename
        )
        
        # 오디오 파일을 출력 디렉토리로 복사
        final_path = audio_dir / output_filename
        if audio_path != final_path:
            shutil.copy(audio_path, final_path)
        
        # 오디오 길이 측정
        audio = MP3(str(final_path))
        duration_ms = int(audio.info.length * 1000)
        
        # Pro 모드 규칙: duration = max(5000, tts_duration)
        scene_duration_ms = max(5000, duration_ms)
        
        status = "✅" if duration_ms <= 5000 else "⚠️ (초과)"
        print(f"{status} {scene_id}: {text[:30]}... → {duration_ms}ms (씬: {scene_duration_ms}ms)")
        
        tts_results.append({
            'scene_id': scene_id,
            'audio_path': str(final_path),
            'tts_duration_ms': duration_ms,
            'duration_ms': scene_duration_ms
        })
        
    except Exception as e:
        print(f"❌ {scene_id}: TTS 실패 - {e}")
        tts_results.append({
            'scene_id': scene_id,
            'audio_path': None,
            'duration_ms': 5000
        })

print(f"\n총 {len(tts_results)}개 TTS 생성 완료")

In [ ]:
# TTS 결과를 plot_data에 반영
for result in tts_results:
    for scene in plot_data['scenes']:
        if scene['scene_id'] == result['scene_id']:
            scene['audio_url'] = result.get('audio_path')
            scene['tts_duration_ms'] = result.get('tts_duration_ms')
            scene['duration_ms'] = result.get('duration_ms', 5000)
            break

# 업데이트된 plot.json 저장
with open(plot_path, 'w', encoding='utf-8') as f:
    json.dump(plot_data, f, indent=2, ensure_ascii=False)

print("✅ plot.json 업데이트 완료 (TTS 정보 추가)")

### 4.2 BGM 생성

In [ ]:
# BGM 생성 (ElevenLabs Sound Effects)
bgm_prompt = plot_data.get('bgm_prompt', 'peaceful ambient background music')
print(f"BGM Prompt: {bgm_prompt}")

# 총 영상 길이 계산
total_duration_ms = sum(scene.get('duration_ms', 5000) for scene in plot_data['scenes'])
print(f"총 영상 길이: {total_duration_ms}ms ({total_duration_ms/1000}초)")

# BGM 생성 (실제로는 composer_task에서 처리)
# 여기서는 스킵하고 테스트용 더미 사용
bgm_path = audio_dir / "global_bgm.mp3"
print(f"\n⚠️ BGM 생성은 composer_task에서 처리됩니다.")
print(f"   예상 출력 경로: {bgm_path}")

### 4.3 이미지 생성 (Start/End Frame)

In [ ]:
# 캐릭터 변수 치환 함수
def substitute_character_vars(prompt: str, characters: list) -> str:
    """Replace {char_1}, {char_2} etc. with actual character descriptions."""
    result = prompt
    for char in characters:
        char_id = char.get('char_id', '')
        description = char.get('description', char.get('name', ''))
        # {char_1} 형식 치환
        result = result.replace(f"{{{char_id}}}", description)
    return result

# 테스트
test_prompt = "{char_1}이 온천에서 휴식을 취하고 있다"
substituted = substitute_character_vars(test_prompt, characters_data['characters'])
print(f"원본: {test_prompt}")
print(f"치환: {substituted}")

In [ ]:
# 이미지 생성 (Gemini Flash)
from app.providers.image.gemini_client import GeminiImageClient

image_client = GeminiImageClient(api_key=settings.GEMINI_API_KEY)
print("✅ Gemini Image 클라이언트 초기화")

# 아트 스타일
art_style = SPEC.get('art_style', '지브리 스타일 애니메이션')
print(f"Art Style: {art_style}")

In [ ]:
# 각 씬별 Start/End 이미지 생성
# 최적화: 이전 씬의 end == 현재 씬의 start면 이미지 재사용

print("=== 이미지 생성 ===")
image_results = []

# 이전 씬의 end 이미지 캐시
prev_end_image_path = None
prev_end_prompt = None

for i, scene in enumerate(plot_data.get('scenes', [])):
    scene_id = scene['scene_id']
    
    # 캐릭터 변수 치환
    start_prompt_raw = scene.get('start_frame_prompt', '')
    end_prompt_raw = scene.get('end_frame_prompt', '')
    
    start_prompt = substitute_character_vars(start_prompt_raw, characters_data['characters'])
    end_prompt = substitute_character_vars(end_prompt_raw, characters_data['characters'])
    
    # 아트 스타일 추가
    start_full = f"{start_prompt}, {art_style}"
    end_full = f"{end_prompt}, {art_style}"
    
    print(f"\n--- {scene_id} ---")
    
    try:
        # Start 이미지: 이전 씬의 end와 동일하면 재사용
        if i > 0 and start_prompt_raw == prev_end_prompt and prev_end_image_path:
            start_path = prev_end_image_path
            print(f"  ♻️ Start 이미지 재사용: {Path(start_path).name}")
        else:
            start_path = image_client.generate_image(
                prompt=start_full,
                output_path=str(OUTPUT_DIR / f"{scene_id}_start.png"),
                aspect_ratio="9:16"
            )
            print(f"  ✅ Start 이미지 생성: {Path(start_path).name}")
        
        # End 이미지 생성 (항상 새로 생성)
        end_path = image_client.generate_image(
            prompt=end_full,
            output_path=str(OUTPUT_DIR / f"{scene_id}_end.png"),
            aspect_ratio="9:16"
        )
        print(f"  ✅ End 이미지 생성: {Path(end_path).name}")
        
        # 캐시 업데이트
        prev_end_image_path = end_path
        prev_end_prompt = end_prompt_raw
        
        image_results.append({
            'scene_id': scene_id,
            'start_image_url': start_path,
            'end_image_url': end_path
        })
        
    except Exception as e:
        print(f"  ❌ 이미지 생성 실패: {e}")
        image_results.append({
            'scene_id': scene_id,
            'start_image_url': None,
            'end_image_url': None
        })

# 이미지 생성 통계
total_images = len(image_results) * 2
reused_images = sum(1 for i, r in enumerate(image_results) 
                   if i > 0 and r.get('start_image_url') == image_results[i-1].get('end_image_url'))
print(f"\n총 이미지: {total_images}개 (재사용: {reused_images}개, 신규: {total_images - reused_images}개)")

In [ ]:
# 이미지 결과를 plot_data에 반영
for result in image_results:
    for scene in plot_data['scenes']:
        if scene['scene_id'] == result['scene_id']:
            scene['start_image_url'] = result.get('start_image_url')
            scene['end_image_url'] = result.get('end_image_url')
            break

# 업데이트된 plot.json 저장
with open(plot_path, 'w', encoding='utf-8') as f:
    json.dump(plot_data, f, indent=2, ensure_ascii=False)

print("✅ plot.json 업데이트 완료 (이미지 경로 추가)")

## 5. Kling AI 영상 생성

### 핵심 규칙
- 입력: start_image + end_image
- 출력: 5초 영상 클립
- API: Kling AI (Image-to-Video)

In [ ]:
# Kling AI 클라이언트 테스트
# API 문서: https://docs.qingque.cn/d/home/eZQDSs3m5bZHj_LXo2aQ3gLPN

print("=== Kling AI 설정 확인 ===")
print(f"KLING_ACCESS_KEY: {'✅ Set' if os.getenv('KLING_ACCESS_KEY') else '❌ Not set'}")
print(f"KLING_SECRET_KEY: {'✅ Set' if os.getenv('KLING_SECRET_KEY') else '❌ Not set'}")

# Kling API 엔드포인트
KLING_API_BASE = "https://api.klingai.com"
KLING_VIDEO_ENDPOINT = f"{KLING_API_BASE}/v1/videos/image2video"
KLING_TASK_ENDPOINT = f"{KLING_API_BASE}/v1/videos/image2video"  # GET for status

In [ ]:
import time
import base64
import jwt
import httpx

class KlingVideoClient:
    """
    Kling AI Image-to-Video 클라이언트
    
    API 흐름:
    1. JWT 토큰 생성 (access_key + secret_key)
    2. POST /v1/videos/image2video → task_id 반환
    3. GET /v1/videos/image2video/{task_id} → 폴링으로 완료 대기
    4. 완료 시 video URL 반환
    """
    
    BASE_URL = "https://api.klingai.com"
    
    def __init__(self, access_key: str, secret_key: str):
        self.access_key = access_key
        self.secret_key = secret_key
        self.client = httpx.Client(timeout=120.0)
    
    def _generate_jwt_token(self) -> str:
        """JWT 토큰 생성 (30분 유효)"""
        now = int(time.time())
        payload = {
            "iss": self.access_key,
            "exp": now + 1800,  # 30분
            "nbf": now - 5
        }
        token = jwt.encode(payload, self.secret_key, algorithm="HS256")
        return token
    
    def _get_headers(self) -> dict:
        return {
            "Authorization": f"Bearer {self._generate_jwt_token()}",
            "Content-Type": "application/json"
        }
    
    def _image_to_base64(self, image_path: str) -> str:
        """이미지를 base64로 인코딩"""
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    
    def generate_video(
        self,
        start_image: str,
        end_image: str = None,
        prompt: str = "",
        duration: str = "5",  # "5" 또는 "10"
        mode: str = "std",  # "std" (표준) 또는 "pro" (고품질)
        output_path: str = None
    ) -> str:
        """
        Image-to-Video 생성
        
        Args:
            start_image: 시작 이미지 경로
            end_image: 종료 이미지 경로 (선택)
            prompt: 동작 설명 프롬프트
            duration: 영상 길이 ("5" 또는 "10")
            mode: 품질 모드 ("std" 또는 "pro")
            output_path: 저장 경로
            
        Returns:
            생성된 영상 경로
        """
        # 1. 요청 페이로드 구성
        payload = {
            "model_name": "kling-v1",
            "mode": mode,
            "duration": duration,
            "image": self._image_to_base64(start_image),
        }
        
        if prompt:
            payload["prompt"] = prompt
            
        if end_image:
            payload["image_tail"] = self._image_to_base64(end_image)
        
        # 2. 영상 생성 요청
        print(f"  📤 Kling API 요청 중...")
        response = self.client.post(
            f"{self.BASE_URL}/v1/videos/image2video",
            headers=self._get_headers(),
            json=payload
        )
        response.raise_for_status()
        result = response.json()
        
        if result.get("code") != 0:
            raise Exception(f"Kling API error: {result.get('message')}")
        
        task_id = result["data"]["task_id"]
        print(f"  📋 Task ID: {task_id}")
        
        # 3. 폴링으로 완료 대기
        video_url = self._poll_task(task_id)
        
        # 4. 영상 다운로드
        if output_path:
            self._download_video(video_url, output_path)
            return output_path
        
        return video_url
    
    def _poll_task(self, task_id: str, max_attempts: int = 60, interval: int = 10) -> str:
        """태스크 완료 대기 (폴링)"""
        for attempt in range(max_attempts):
            response = self.client.get(
                f"{self.BASE_URL}/v1/videos/image2video/{task_id}",
                headers=self._get_headers()
            )
            response.raise_for_status()
            result = response.json()
            
            if result.get("code") != 0:
                raise Exception(f"Kling API error: {result.get('message')}")
            
            status = result["data"]["task_status"]
            
            if status == "succeed":
                video_url = result["data"]["task_result"]["videos"][0]["url"]
                print(f"  ✅ 영상 생성 완료!")
                return video_url
            elif status == "failed":
                raise Exception(f"영상 생성 실패: {result['data'].get('task_status_msg')}")
            else:
                # processing, submitted 등
                print(f"  ⏳ 생성 중... ({attempt + 1}/{max_attempts}) - {status}")
                time.sleep(interval)
        
        raise TimeoutError(f"영상 생성 타임아웃 ({max_attempts * interval}초)")
    
    def _download_video(self, url: str, output_path: str):
        """영상 다운로드"""
        print(f"  📥 영상 다운로드 중...")
        response = self.client.get(url)
        response.raise_for_status()
        
        with open(output_path, "wb") as f:
            f.write(response.content)
        
        print(f"  💾 저장 완료: {output_path}")

print("✅ KlingVideoClient 클래스 정의 완료")

In [ ]:
# Kling AI 클라이언트 초기화 및 영상 생성
KLING_ACCESS_KEY = os.getenv("KLING_ACCESS_KEY")
KLING_SECRET_KEY = os.getenv("KLING_SECRET_KEY")

if KLING_ACCESS_KEY and KLING_SECRET_KEY:
    kling_client = KlingVideoClient(
        access_key=KLING_ACCESS_KEY,
        secret_key=KLING_SECRET_KEY
    )
    print("✅ Kling AI 클라이언트 초기화 완료")
else:
    kling_client = None
    print("⚠️ Kling API 키가 설정되지 않았습니다. .env에 추가해주세요:")
    print("   KLING_ACCESS_KEY=your_access_key")
    print("   KLING_SECRET_KEY=your_secret_key")

In [ ]:
# 각 씬별 Kling 영상 생성
print("=== Kling AI 영상 생성 ===")

video_results = []

if not kling_client:
    print("❌ Kling 클라이언트가 없어 스킵합니다.")
else:
    for scene in plot_data.get('scenes', []):
        scene_id = scene['scene_id']
        start_image = scene.get('start_image_url')
        end_image = scene.get('end_image_url')
        
        if not start_image or not Path(start_image).exists():
            print(f"⚠️ {scene_id}: 시작 이미지 없음, 스킵")
            video_results.append({'scene_id': scene_id, 'video_url': None})
            continue
        
        video_path = str(OUTPUT_DIR / f"{scene_id}_video.mp4")
        
        print(f"\n--- {scene_id} ---")
        print(f"  Start: {Path(start_image).name}")
        print(f"  End: {Path(end_image).name if end_image else 'N/A'}")
        
        try:
            # Kling 영상 생성 (5초)
            result_path = kling_client.generate_video(
                start_image=start_image,
                end_image=end_image,  # 종료 이미지로 자연스러운 전환
                prompt="",  # 프롬프트 없이 이미지 기반 생성
                duration="5",
                mode="std",  # std: 빠름, pro: 고품질
                output_path=video_path
            )
            
            video_results.append({
                'scene_id': scene_id,
                'video_url': result_path
            })
            
        except Exception as e:
            print(f"  ❌ 영상 생성 실패: {e}")
            video_results.append({
                'scene_id': scene_id,
                'video_url': None,
                'error': str(e)
            })

print(f"\n총 {len([v for v in video_results if v.get('video_url')])}개 영상 생성 완료")

In [ ]:
# FFmpeg 기반 합성 로직 - Pro Mode
import subprocess
import tempfile

def get_video_duration(video_path: str) -> float:
    """FFprobe로 영상 길이 조회"""
    cmd = [
        "ffprobe", "-v", "error",
        "-show_entries", "format=duration",
        "-of", "default=noprint_wrappers=1:nokey=1",
        video_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return float(result.stdout.strip())

def get_audio_duration(audio_path: str) -> float:
    """FFprobe로 오디오 길이 조회"""
    cmd = [
        "ffprobe", "-v", "error",
        "-show_entries", "format=duration",
        "-of", "default=noprint_wrappers=1:nokey=1",
        audio_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return float(result.stdout.strip())

def compose_scene_ffmpeg(
    video_path: str,
    audio_path: str,
    text: str,
    output_path: str,
    font_path: str = "/System/Library/Fonts/AppleSDGothicNeo.ttc"
) -> str:
    """
    단일 씬 합성 (FFmpeg)
    - 영상 + TTS 오버레이
    - TTS > 영상이면 프레임 정지
    - 자막 추가
    """
    video_duration = get_video_duration(video_path)
    
    # TTS 길이 확인
    if audio_path and Path(audio_path).exists():
        audio_duration = get_audio_duration(audio_path)
    else:
        audio_duration = 0
    
    # 최종 씬 길이 = max(영상, TTS)
    scene_duration = max(video_duration, audio_duration)
    
    # FFmpeg 필터 구성
    filters = []
    
    # 1. 프레임 정지 (TTS가 영상보다 길 때)
    if audio_duration > video_duration:
        freeze_duration = audio_duration - video_duration
        # tpad: 마지막 프레임 정지
        filters.append(f"tpad=stop_mode=clone:stop_duration={freeze_duration}")
        print(f"  ⏸️ 프레임 정지: {freeze_duration:.2f}초")
    
    # 2. 자막 추가
    if text:
        # 특수문자 이스케이프
        escaped_text = text.replace("'", "'\\''").replace(":", "\\:")
        filters.append(
            f"drawtext=text='{escaped_text}':"
            f"fontfile='{font_path}':"
            f"fontsize=40:fontcolor=white:"
            f"borderw=2:bordercolor=black:"
            f"x=(w-text_w)/2:y=h-100"
        )
    
    # FFmpeg 명령 구성
    filter_str = ",".join(filters) if filters else "null"
    
    cmd = ["ffmpeg", "-y"]
    
    # 입력: 영상
    cmd.extend(["-i", video_path])
    
    # 입력: 오디오 (있으면)
    if audio_path and Path(audio_path).exists():
        cmd.extend(["-i", audio_path])
    
    # 비디오 필터
    cmd.extend(["-vf", filter_str])
    
    # 오디오 매핑
    if audio_path and Path(audio_path).exists():
        cmd.extend(["-map", "0:v", "-map", "1:a"])
    else:
        cmd.extend(["-an"])  # 오디오 없음
    
    # 출력 설정
    cmd.extend([
        "-c:v", "libx264",
        "-preset", "fast",
        "-crf", "23",
        "-c:a", "aac",
        "-b:a", "128k",
        output_path
    ])
    
    subprocess.run(cmd, check=True, capture_output=True)
    return output_path

print("✅ compose_scene_ffmpeg 함수 정의 완료")

In [ ]:
def compose_final_video_ffmpeg(
    plot_data: dict,
    output_dir: Path,
    bgm_path: Path = None,
    bgm_volume: float = 0.3
) -> str:
    """
    전체 영상 합성 (FFmpeg)
    
    1. 각 씬별 합성 (영상 + TTS + 자막)
    2. 모든 씬 연결 (concat)
    3. BGM 믹싱
    """
    temp_dir = output_dir / "temp"
    temp_dir.mkdir(exist_ok=True)
    
    # 1. 각 씬별 합성
    print("=== Step 1: 씬별 합성 ===")
    scene_files = []
    
    for scene in plot_data.get('scenes', []):
        scene_id = scene['scene_id']
        video_url = scene.get('video_url')
        audio_url = scene.get('audio_url')
        text = scene.get('text', '')
        
        if not video_url or not Path(video_url).exists():
            print(f"⚠️ {scene_id}: 영상 없음, 스킵")
            continue
        
        scene_output = str(temp_dir / f"{scene_id}_composed.mp4")
        
        print(f"\n--- {scene_id} ---")
        try:
            compose_scene_ffmpeg(
                video_path=video_url,
                audio_path=audio_url,
                text=text,
                output_path=scene_output
            )
            scene_files.append(scene_output)
            print(f"  ✅ 합성 완료: {Path(scene_output).name}")
        except Exception as e:
            print(f"  ❌ 합성 실패: {e}")
    
    if not scene_files:
        raise ValueError("합성할 씬이 없습니다")
    
    # 2. 씬 연결 (concat)
    print("\n=== Step 2: 씬 연결 ===")
    concat_list = temp_dir / "concat_list.txt"
    with open(concat_list, 'w') as f:
        for scene_file in scene_files:
            f.write(f"file '{scene_file}'\n")
    
    concat_output = str(temp_dir / "concat_output.mp4")
    cmd = [
        "ffmpeg", "-y",
        "-f", "concat",
        "-safe", "0",
        "-i", str(concat_list),
        "-c", "copy",
        concat_output
    ]
    subprocess.run(cmd, check=True, capture_output=True)
    print(f"✅ {len(scene_files)}개 씬 연결 완료")
    
    # 3. BGM 믹싱
    final_output = str(output_dir / "final_video.mp4")
    
    if bgm_path and bgm_path.exists():
        print("\n=== Step 3: BGM 믹싱 ===")
        
        # 영상 길이 조회
        video_duration = get_video_duration(concat_output)
        
        # BGM + TTS 믹싱
        cmd = [
            "ffmpeg", "-y",
            "-i", concat_output,
            "-i", str(bgm_path),
            "-filter_complex",
            f"[1:a]aloop=loop=-1:size=2e+09,atrim=0:{video_duration},volume={bgm_volume}[bgm];"
            f"[0:a][bgm]amix=inputs=2:duration=first:dropout_transition=2[aout]",
            "-map", "0:v",
            "-map", "[aout]",
            "-c:v", "copy",
            "-c:a", "aac",
            "-b:a", "192k",
            final_output
        ]
        subprocess.run(cmd, check=True, capture_output=True)
        print(f"✅ BGM 믹싱 완료 (볼륨: {int(bgm_volume*100)}%)")
    else:
        # BGM 없으면 그냥 복사
        shutil.copy(concat_output, final_output)
        print("⚠️ BGM 없음, TTS만 포함")
    
    # 4. 임시 파일 정리 (선택)
    # shutil.rmtree(temp_dir)
    
    print(f"\n🎬 최종 영상: {final_output}")
    return final_output

print("✅ compose_final_video_ffmpeg 함수 정의 완료")

In [ ]:
# 최종 합성 실행
print("=== 최종 영상 합성 (FFmpeg) ===\n")

# BGM 경로 (있으면)
bgm_path = audio_dir / "global_bgm.mp3"

try:
    final_video = compose_final_video_ffmpeg(
        plot_data=plot_data,
        output_dir=OUTPUT_DIR,
        bgm_path=bgm_path if bgm_path.exists() else None,
        bgm_volume=0.3
    )
    print(f"\n✅ 최종 영상 생성 완료: {final_video}")
except Exception as e:
    print(f"\n❌ 합성 실패: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 영상 결과를 plot_data에 반영
for result in video_results:
    for scene in plot_data['scenes']:
        if scene['scene_id'] == result['scene_id']:
            scene['video_url'] = result.get('video_url')
            break

# 업데이트된 plot.json 저장
with open(plot_path, 'w', encoding='utf-8') as f:
    json.dump(plot_data, f, indent=2, ensure_ascii=False)

print("✅ plot.json 업데이트 완료 (영상 경로 추가)")

# 현재 상태 확인
print("\n=== 씬별 에셋 현황 ===")
for scene in plot_data['scenes']:
    has_tts = "✅" if scene.get('audio_url') else "❌"
    has_video = "✅" if scene.get('video_url') else "❌"
    print(f"  {scene['scene_id']}: TTS{has_tts} Video{has_video}")

## 6. 최종 합성

### 합성 규칙
1. 각 씬별 Kling 영상 + TTS 오버레이
2. TTS가 영상보다 길면 영상 마지막 프레임 정지
3. 자막 추가
4. 전역 BGM 오버레이 (볼륨 30%)
5. 모든 씬 연결 → final_video.mp4

In [ ]:
# MoviePy 기반 합성 로직 - 실제 구현 가능한 코드
print("=== 합성 로직 (Pro Mode) ===")
print("")

def compose_pro_video(plot_data: dict, output_dir: Path, bgm_path: Path = None):
    """
    Pro 모드 영상 합성 함수
    
    핵심 규칙:
    1. TTS > 영상 길이면 마지막 프레임 정지
    2. 자막 하단 배치
    3. BGM 30% 볼륨으로 믹싱
    """
    from moviepy import (
        VideoFileClip, ImageClip, TextClip, AudioFileClip,
        CompositeVideoClip, CompositeAudioClip, concatenate_videoclips
    )
    
    final_clips = []
    
    for scene in plot_data['scenes']:
        scene_id = scene['scene_id']
        video_url = scene.get('video_url')
        audio_url = scene.get('audio_url')
        tts_duration_ms = scene.get('tts_duration_ms', 0)
        text = scene.get('text', '')
        
        if not video_url or not Path(video_url).exists():
            print(f"⚠️ {scene_id}: 영상 없음, 스킵")
            continue
        
        # 1. Kling 영상 로드 (5초)
        video_clip = VideoFileClip(video_url)
        video_duration = video_clip.duration
        
        # 2. TTS 길이 체크 및 프레임 정지
        tts_duration = tts_duration_ms / 1000 if tts_duration_ms else 0
        
        if tts_duration > video_duration:
            # 마지막 프레임 정지 (freeze frame)
            freeze_duration = tts_duration - video_duration
            last_frame = video_clip.get_frame(video_duration - 0.1)
            freeze_clip = ImageClip(last_frame).with_duration(freeze_duration)
            video_clip = concatenate_videoclips([video_clip, freeze_clip])
            print(f"  {scene_id}: 프레임 정지 {freeze_duration:.1f}초 추가")
        
        # 3. TTS 오디오 오버레이
        if audio_url and Path(audio_url).exists():
            tts_audio = AudioFileClip(audio_url)
            video_clip = video_clip.with_audio(tts_audio)
        
        # 4. 자막 추가
        if text:
            subtitle = TextClip(
                text=text,
                font='NanumGothic-Bold',  # 한글 폰트
                font_size=40,
                color='white',
                stroke_color='black',
                stroke_width=2,
                method='caption',
                size=(video_clip.w - 40, None)  # 좌우 20px 여백
            )
            subtitle = subtitle.with_position(('center', video_clip.h - 100))
            subtitle = subtitle.with_duration(video_clip.duration)
            video_clip = CompositeVideoClip([video_clip, subtitle])
        
        final_clips.append(video_clip)
        print(f"  ✅ {scene_id}: {video_clip.duration:.1f}초")
    
    if not final_clips:
        raise ValueError("합성할 영상 클립이 없습니다")
    
    # 5. 모든 씬 연결
    final_video = concatenate_videoclips(final_clips, method='compose')
    print(f"\n총 영상 길이: {final_video.duration:.1f}초")
    
    # 6. BGM 오버레이 (볼륨 30%)
    if bgm_path and bgm_path.exists():
        bgm = AudioFileClip(str(bgm_path))
        # BGM이 영상보다 짧으면 루프
        if bgm.duration < final_video.duration:
            bgm = bgm.loop(duration=final_video.duration)
        else:
            bgm = bgm.subclipped(0, final_video.duration)
        
        bgm = bgm.with_volume_scaled(0.3)  # 30% 볼륨
        
        # 기존 TTS 오디오와 BGM 믹스
        if final_video.audio:
            final_audio = CompositeAudioClip([final_video.audio, bgm])
            final_video = final_video.with_audio(final_audio)
        else:
            final_video = final_video.with_audio(bgm)
        
        print("✅ BGM 믹싱 완료 (30% 볼륨)")
    
    # 7. 최종 출력
    output_path = output_dir / 'final_video.mp4'
    final_video.write_videofile(
        str(output_path),
        fps=30,
        codec='libx264',
        audio_codec='aac',
        threads=4
    )
    
    # 리소스 정리
    final_video.close()
    for clip in final_clips:
        clip.close()
    
    return output_path

print("✅ compose_pro_video 함수 정의 완료")
print("")
print("사용법:")
print("  output = compose_pro_video(plot_data, OUTPUT_DIR, bgm_path)")

## 7. 결과 요약

In [ ]:
## 8. TODO 체크리스트

### 노트북 구현 완료 ✅
- [x] 프롬프트 증강 (enhance_prompt) - pro 모드 지원
- [x] 플롯 생성 (generate_plot_pro_mode) - characters.json + plot.json
- [x] TTS 생성 (MiniMax/ElevenLabs)
- [x] BGM 생성 (composer_task 재사용)
- [x] 이미지 생성 (Gemini Flash) - 이미지 재사용 최적화 포함
- [x] **Kling AI 클라이언트** - KlingVideoClient 클래스
- [x] **FFmpeg 합성 로직** - compose_scene_ffmpeg, compose_final_video_ffmpeg

### 백엔드 통합 필요
- [ ] `app/providers/video/kling_client.py` - 노트북 코드 이식
- [ ] `app/config.py` - KLING_ACCESS_KEY, KLING_SECRET_KEY 추가
- [ ] `app/tasks/designer.py` - designer_task_pro에 Kling 연동
- [ ] `app/tasks/director.py` - director_task_pro FFmpeg 합성

### .env 설정 필요
```bash
KLING_ACCESS_KEY=your_access_key
KLING_SECRET_KEY=your_secret_key
```

## 8. TODO 체크리스트

### 구현 완료
- [x] 프롬프트 증강 (enhance_prompt) - pro 모드 지원
- [x] 플롯 생성 (generate_plot_pro_mode) - characters.json + plot.json
- [x] TTS 생성 (MiniMax/ElevenLabs) - voice_task_pro
- [x] BGM 생성 (composer_task) - 기존 로직 재사용
- [x] 이미지 생성 (Gemini Flash) - designer_task_pro

### 구현 필요
- [ ] **Kling AI 클라이언트** ← 핵심!
  - `app/providers/video/kling_client.py`
  - Image-to-Video API 연동
  - 5초 영상 생성
  
- [ ] **director_task_pro 합성 로직**
  - TTS > 5초 시 영상 프레임 정지
  - 자막 오버레이
  - BGM 믹싱
  - 씬 연결

### 최적화 (Optional)
- [ ] 씬 연결 시 이미지 재사용 (end_image = next_start_image)
- [ ] Kling API 비동기 폴링 (긴 생성 시간 대응)
- [ ] 캐릭터 seed 기반 일관성 유지